In [2]:
import pandas as pd 
import os 
import numpy as np 
from glob import glob 
import yaml

from src.datafactory import dataloader 
from src.model import Generator,Discriminator,weights_init
from src.Network import Encoder,Decoder 
from src.test import metrics 
from src.scheduler import CosineAnnealingWarmupRestarts

import torch 
import torch.nn as nn 
import torch.nn.functional as F 

from sklearn.metrics import roc_auc_score,roc_curve,auc 

cfg = yaml.load(open('./config/default.yaml','r'), Loader=yaml.FullLoader)

device = cfg['TRAIN']['device']

trainloader = dataloader(
    datadir     = './Data/train_ver2.csv',
    window_size = cfg['DATA']['window_size'],
    stride      = cfg['DATA']['stride'],
    batch_size  = cfg['TRAIN']['batchsize'],
    shuffle     = True 
)
testloader = dataloader(
    datadir     = './Data/test_ver2.csv',
    window_size = cfg['DATA']['window_size'],
    stride      = cfg['DATA']['stride'],
    batch_size  = cfg['TRAIN']['batchsize'],
    shuffle     = False  
) 

linear = nn.Sequential(nn.Linear(5120,256),
              nn.BatchNorm1d(256),
              nn.ReLU(),
              nn.Linear(256,1)).to(device)

# build model 
D = Discriminator(
    cfg['MODEL']['in_c'],
    cfg['MODEL']['hidden_c'],
    cfg['MODEL']['latent_c']).apply(weights_init).to(device)
G = Generator(
    cfg['MODEL']['in_c'],
    cfg['MODEL']['hidden_c'],
    cfg['MODEL']['latent_c']).apply(weights_init).to(device)

# build loss function 
bce_criterion = nn.BCELoss()
mse_criterion = nn.MSELoss()

# build optimizer 
optimizerG = torch.optim.Adam(G.parameters(),lr=cfg['TRAIN']['lr'],betas=(0.999,0.999))
optimizerD = torch.optim.Adam(D.parameters(),lr=cfg['TRAIN']['lr'],betas=(0.999,0.999))
schedulerG = CosineAnnealingWarmupRestarts(
            optimizerG, 
            first_cycle_steps = cfg['TRAIN']['epochs'],
            max_lr            = cfg['TRAIN']['lr'],
            min_lr            = cfg['TRAIN']['min_lr'],
            warmup_steps      = int(cfg['TRAIN']['epochs'] * cfg['TRAIN']['warmup_ratio'])
        )
schedulerD = CosineAnnealingWarmupRestarts(
            optimizerD, 
            first_cycle_steps = cfg['TRAIN']['epochs'],
            max_lr            = cfg['TRAIN']['lr'],
            min_lr            = cfg['TRAIN']['min_lr'],
            warmup_steps      = int(cfg['TRAIN']['epochs'] * cfg['TRAIN']['warmup_ratio'])
        )
class LossMeter:
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.loss_g_adv  = 0.0
        self.loss_g_rec  = 0.0
        self.loss_g      = 0.0 
        
        self.loss_d_real = 0.0 
        self.loss_d_fake = 0.0 
        self.loss_d      = 0.0 
        
        self.count = 0 
        
    def update(self,log,n=1):
        self.count += n 
        
        self.loss_g_adv  += log['loss_g_adv']
        self.loss_g_rec  += log['loss_g_rec']
        self.loss_g      += log['loss_g'] 
        
        self.loss_d_real += log['loss_d_real'] 
        self.loss_d_fake += log['loss_d_fake'] 
        self.loss_d      += log['loss_d'] 
        
    def avg(self):
        log = {
            'loss_g_adv'  : self.loss_g_adv/self.count,
            'loss_g_rec'  : self.loss_g_rec/self.count,
            'loss_g'      : self.loss_g    /self.count,
            'loss_d_real' : self.loss_d_real/self.count,
            'loss_d_fake' : self.loss_d_fake/self.count,
            'loss_d'      : self.loss_d    /self.count
            }
        self.reset()
        return log 
    
def updateD(G,D,x,y,bce_criterion):
    real_label = 1 #해당 라벨은 생성한 이미지가 진짜인지 가짜인지 판별하는 라벨 
    fake_label = 0 
    
    G.eval()
    D.train()
    D.zero_grad()
    
    # Inference 
    out_d_real,_ = D(x)
    with torch.no_grad():
        recon_x,_ = G(x)
    out_d_fake,_ = D(recon_x)
    
    loss_d_real = bce_criterion(out_d_real,torch.full((x.shape[0],), real_label, device=device).type(torch.float32))
    loss_d_fake = bce_criterion(out_d_fake,torch.full((x.shape[0],), fake_label,device=device).type(torch.float32))
    
    # loss backward 
    loss_d = loss_d_real + loss_d_fake 
    optimizerD.zero_grad()
    loss_d.backward()
    optimizerD.step()
    return loss_d, loss_d_real,loss_d_fake 
        

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
real_label = 1 #해당 라벨은 생성한 이미지가 진짜인지 가짜인지 판별하는 라벨 
fake_label = 0 

for epoch in range(cfg['TRAIN']['epochs']):
    #train epoch 
    G.train()
    D.train()

    loss_meter = LossMeter() 
    best = 0 
    for i,(x,y) in enumerate(trainloader):
        x,y = x.to(device),y.to(device)
        
        #def  update D 
        G.eval()
        D.train()
        D.zero_grad()
        
        # Train with real 
        out_d_real,_ = D(x)
        loss_d_real = bce_criterion(
                                    out_d_real,
                                    torch.full((x.shape[0],), real_label, device=device).type(torch.float32)
                                    )
        
        # Train with fake 
        with torch.no_grad():
            recon_x,_ = G(x)
        out_d_fake,_ = D(recon_x)
        loss_d_fake = bce_criterion(
                                    out_d_fake,
                                    torch.full((x.shape[0],),fake_label,device=device).type(torch.float32)
                                    )
        
        # loss backward 
        loss_d = loss_d_real + loss_d_fake
        optimizerD.zero_grad()
        loss_d.backward()
        optimizerD.step()
        
        # ! update G 
        G.train()
        D.eval()
        G.zero_grad()
        
        #reconsturction 
        recon_x,latent_z = G(x)
        
        with torch.no_grad():
            _,feat_real = D(x) # original feature
            out_g, feat_fake = D(recon_x) # reconstruction feature 
        
        # loss 
        loss_g_adv = mse_criterion(feat_fake,feat_real) # loss for feature matching 
        loss_g_rec = mse_criterion(recon_x,x) # reconstruction 
        
        # backward 
        loss_g = loss_g_adv + loss_g_rec * 1 # w_adv = 1 
        optimizerG.zero_grad()
        loss_g.backward()
        optimizerG.step()
        
        #logging 
        log = {
        'loss_g_adv' : loss_g_adv ,
        'loss_g_rec' : loss_g_rec ,
        'loss_g'     : loss_g     ,
        'loss_d_real': loss_d_real,
        'loss_d_fake': loss_d_fake,
        'loss_d'     : loss_d     }
        loss_meter.update(log)
        
    #scheduler step 
    #schedulerD.step()
    #schedulerG.step()
    
    # Epoch evaluate 
    log = loss_meter.avg()
    auroc,acc,y_true,y_pred,thr = metrics(G,testloader,device)
    print(f"Epoch:[{epoch}/{cfg['TRAIN']['epochs']}] |G loss : {log['loss_g']:.3f}|D Loss : {log['loss_d']:.3f}|AUROC : {auroc:.3f}|ACC : {acc:.3f}")
    if acc > best:
        torch.save(G,f"{cfg['SAVE']['savedir']}/best_G.pt")        
        torch.save(D,f"{cfg['SAVE']['savedir']}/best_D.pt")
        best = acc 
    
torch.save(G,f"{cfg['SAVE']['savedir']}/last_G.pt")        
torch.save(D,f"{cfg['SAVE']['savedir']}/last_D.pt")

Epoch:[0/100] |G loss : 4.520|D Loss : 0.352|AUROC : 0.645|ACC : 0.459
Epoch:[1/100] |G loss : 5.745|D Loss : 0.090|AUROC : 0.643|ACC : 0.435
Epoch:[2/100] |G loss : 3.749|D Loss : 0.009|AUROC : 0.647|ACC : 0.432
Epoch:[3/100] |G loss : 3.718|D Loss : 0.004|AUROC : 0.667|ACC : 0.432
Epoch:[4/100] |G loss : 3.215|D Loss : 0.004|AUROC : 0.668|ACC : 0.428
Epoch:[5/100] |G loss : 2.924|D Loss : 0.005|AUROC : 0.420|ACC : 0.211
Epoch:[6/100] |G loss : 2.810|D Loss : 0.001|AUROC : 0.361|ACC : 0.175
Epoch:[7/100] |G loss : 2.484|D Loss : 0.001|AUROC : 0.342|ACC : 0.114
Epoch:[8/100] |G loss : 2.113|D Loss : 0.004|AUROC : 0.548|ACC : 0.270
Epoch:[9/100] |G loss : 1.805|D Loss : 0.003|AUROC : 0.610|ACC : 0.370
Epoch:[10/100] |G loss : 1.534|D Loss : 0.002|AUROC : 0.627|ACC : 0.404
Epoch:[11/100] |G loss : 1.351|D Loss : 0.000|AUROC : 0.635|ACC : 0.420
Epoch:[12/100] |G loss : 1.232|D Loss : 0.001|AUROC : 0.640|ACC : 0.399
Epoch:[13/100] |G loss : 1.155|D Loss : 0.001|AUROC : 0.650|ACC : 0.424
Ep

In [4]:
real_label = 1 #해당 라벨은 생성한 이미지가 진짜인지 가짜인지 판별하는 라벨 
fake_label = 0 

for epoch in range(cfg['TRAIN']['epochs']):
    #train epoch 
    G.train()
    D.train()

    loss_meter = LossMeter() 
    best = 0 
    for i,(x,y) in enumerate(trainloader):
        x,y = x.to(device),y.to(device)
        
        #def  update D 
        G.eval()
        D.train()
        D.zero_grad()
        
        # Train with real 
        out_d_real,_ = D(x)
        loss_d_real = bce_criterion(
                                    out_d_real,
                                    torch.full((x.shape[0],), real_label, device=device).type(torch.float32)
                                    )
        
        # Train with fake 
        with torch.no_grad():
            recon_x,_ = G(x)
        out_d_fake,_ = D(recon_x)
        loss_d_fake = bce_criterion(
                                    out_d_fake,
                                    torch.full((x.shape[0],),fake_label,device=device).type(torch.float32)
                                    )
        
        # loss backward 
        loss_d = loss_d_real + loss_d_fake
        optimizerD.zero_grad()
        loss_d.backward()
        optimizerD.step()
        
        # ! update G 
        G.train()
        D.eval()
        G.zero_grad()
        
        #reconsturction 
        recon_x,latent_z = G(x)
        
        with torch.no_grad():
            _,feat_real = D(x) # original feature
            out_g, feat_fake = D(recon_x) # reconstruction feature 
        
        # loss 
        loss_g_adv = mse_criterion(feat_fake,feat_real) # loss for feature matching 
        loss_g_rec = mse_criterion(recon_x,x) # reconstruction 
        
        # backward 
        loss_g = loss_g_adv + loss_g_rec * 1 # w_adv = 1 
        optimizerG.zero_grad()
        loss_g.backward()
        optimizerG.step()
        
        #logging 
        log = {
        'loss_g_adv' : loss_g_adv ,
        'loss_g_rec' : loss_g_rec ,
        'loss_g'     : loss_g     ,
        'loss_d_real': loss_d_real,
        'loss_d_fake': loss_d_fake,
        'loss_d'     : loss_d     }
        loss_meter.update(log)
        
    #scheduler step 
    #schedulerD.step()
    #schedulerG.step()
    
    # Epoch evaluate 
    log = loss_meter.avg()
    auroc,acc,y_true,y_pred,thr = metrics(G,testloader,device)
    print(f"Epoch:[{epoch}/{cfg['TRAIN']['epochs']}] |G loss : {log['loss_g']:.3f}|D Loss : {log['loss_d']:.3f}|AUROC : {auroc:.3f}|ACC : {acc:.3f}")
    if acc > best:
        torch.save(G,f"{cfg['SAVE']['savedir']}/best_G.pt")        
        torch.save(D,f"{cfg['SAVE']['savedir']}/best_D.pt")
        best = acc 
    
torch.save(G,f"{cfg['SAVE']['savedir']}/last_G.pt")        
torch.save(D,f"{cfg['SAVE']['savedir']}/last_D.pt")

Epoch:[0/100] |G loss : 0.477|D Loss : 0.000|AUROC : 0.738|ACC : 0.569
Epoch:[1/100] |G loss : 0.473|D Loss : 0.000|AUROC : 0.738|ACC : 0.571
Epoch:[2/100] |G loss : 0.473|D Loss : 0.000|AUROC : 0.740|ACC : 0.575
Epoch:[3/100] |G loss : 0.480|D Loss : 0.000|AUROC : 0.746|ACC : 0.583
Epoch:[4/100] |G loss : 0.490|D Loss : 0.000|AUROC : 0.752|ACC : 0.595
Epoch:[5/100] |G loss : 0.505|D Loss : 0.000|AUROC : 0.757|ACC : 0.604
Epoch:[6/100] |G loss : 0.517|D Loss : 0.000|AUROC : 0.758|ACC : 0.616
Epoch:[7/100] |G loss : 0.528|D Loss : 0.000|AUROC : 0.757|ACC : 0.628
Epoch:[8/100] |G loss : 0.539|D Loss : 0.000|AUROC : 0.758|ACC : 0.638
Epoch:[9/100] |G loss : 0.546|D Loss : 0.000|AUROC : 0.759|ACC : 0.638
Epoch:[10/100] |G loss : 0.556|D Loss : 0.000|AUROC : 0.762|ACC : 0.637
Epoch:[11/100] |G loss : 0.560|D Loss : 0.000|AUROC : 0.763|ACC : 0.628
Epoch:[12/100] |G loss : 0.561|D Loss : 0.000|AUROC : 0.762|ACC : 0.614
Epoch:[13/100] |G loss : 0.548|D Loss : 0.000|AUROC : 0.761|ACC : 0.605
Ep

KeyboardInterrupt: 

# Evaluate 

In [11]:
y_list = [] 
score_list = [] 
G.eval()
for x,y in testloader:
    x = x.to(device)
    
    with torch.no_grad():
        recon_x,_ = G(x)
    anomaly_score = torch.mean(torch.pow((x-recon_x),2),dim=(1,2)).detach().cpu().numpy()
    
    score_list.extend(anomaly_score)
    y_list.extend(y.detach().cpu().numpy())

In [23]:
from sklearn.metrics import precision_score,f1_score,accuracy_score,recall_score
thres = np.percentile(score_list,80)
result = [] 
for me in [precision_score,f1_score,accuracy_score,recall_score]:
    re = me(y_list,pd.Series(score_list).apply(lambda x : 1 if x > thres else 0).values)
    result.append(re)
print(result)

[0.7669576897246474, 0.6383751805096195, 0.8262183519509302, 0.5467166679964892]


In [4]:


def updateG(G,D,x,y,bce_criterion,mse_criterion):
    G.train()
    D.eval()
    G.zero_grad()
    
    # reconstruction 
    recon_x,latent_z = G(x)
    
    with torch.no_grad():
        _,feat_real = D(x)
        out_g, feat_fake = D(recon_x)
        
    # loss 
    loss_g_adv = mse_criterion(feat_fake,feat_real)
    loss_g_rec = mse_criterion(recon_x,x)
    
    # backward 
    loss_g = loss_g_adv + loss_g_rec 
    optimizerG.zero_grad()
    loss_g.backward()
    optimizerG.step()
    
    return loss_g,loss_g_adv,loss_g_rec